# 👑 RL and the $N$-Queen Problem
### Viviana Márquez
#### June 27th, 2019

# Introduction

• It's a classical problem in combinatorics proposed in 1848.

• People are still working on this problem as of today.

# Introduction

• **Question:** 

How many ways can you place $n$-queens in an $n \times n$ chessboard so that no two queens threaten each other?

<center><img src="https://qph.fs.quoracdn.net/main-qimg-04a0f0702f86ecb8931bdd75af9e9a55"></center>

# Example: $n=1$

<center><img src="images/1queen.png"></center>

<center><b>1 solution!</b></center>

# Example: $n=2$

<center><img src="images/2queens.png"></center>

<center><b>0 solutions!</b></center>

# Example: $n=3$

<center><img src="images/3queens.png"></center>

<center><b>0 solutions!</b></center>

# Example: $n=4$

<center><img src="images/4queens.png"></center>

<center><b>2 solutions!</b></center>

# Known solutions...

<img src="images/all.png">

• Highest-order known solution as of today: $n=27$ [(more info here)](https://github.com/preusser/q27).

# Example: $n=8$

<center><img src="images/queens.png" height='0.1'></center>

<center><b>92 solutions!</b></center>

# Example: $n=8$

<center><img src="images/queens.png" height='0.1'></center>

<center>92 solutions!</center><br>
<center><b>Out of $\binom{64}{8}$ possible boards!</b></center>

# Example: $n=8$

<center><img src="images/queens.png" height='0.1'></center>

<center>92 solutions!</center><br>
<center><b>Out of 4,426,165,368 possible boards!</b></center>

# 🔎 Scope of this project:

<br>
<center>Find <it>a</it> solution (queen placement) using Reinforcement Learning.</center>

# The $n$-queens problem as a Reinforcement Learning problem:

<br>
<center><b>Research question:</b> Can I make a RL agent learn how to find solutions to the n-queen problem?</center>
<br>

• **Agent:** Machine

• **Environment:** Chessboard

• **State:** Placement of the queens

• **Action:** Coordinates of the next queen to place

• **Reward:** 1 for a valid placement, 0 for a non-valid placement

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
class Queens():
    def __init__(self, n):
        self.n = n
        self.board = np.array([["*" for _ in range(n)] for _ in range(n)])
        self.state = ["*" for _ in range(n)]
        self.available = np.array([["O" for _ in range(n)] for _ in range(n)])
        self.allowed_moves = list(zip(*np.where(self.available == "O")))
    
    def print_board(self):
        """
        Print current state of the game.
        """
        return print(pd.DataFrame(self.board))
    
    def print_available(self):
        """
        Prints available places.
        """
        return print(pd.DataFrame(self.available))
    
    def is_sol(self):
        """
        Check if current state is a solution.
        """
        diagonals = [self.board.diagonal(i) for i in range(-self.n+1,self.n)] + \
                    [self.board[::-1,:].diagonal(i) for i in range(-self.n+1,self.n)]
        for d in diagonals:
            if list(d).count("Q") > 1:
                return False
        if "*" in self.state:
            return False
        return True
    
    def put_queen(self,x,y):
        """
        Place a queen
        """
        if self.available[y][x] == "X": return "Not a valid movement. Try again!"
        self.state[x] = y
        self.board[y][x] = "Q"
        self.update_available(x,y)
        return self.print_board()
    
    def update_available(self,x,y):
        self.available[y][x] = "X"
        self.available[y,:] = "X"
        self.available[:,x] = "X"
        
        aux = np.arange(self.n)
        diag_1 = aux[::-1,None] == aux + self.n - y - x -1
        diag_2 = aux[:,None] == aux + y - x
        self.available[diag_1|diag_2] = "X"
        self.allowed_moves = list(zip(*np.where(self.available == "O")))
          
    def random_board(self):
        """
        Return a random placement of the queens.
        """
        self.restart()
        self.state = np.random.permutation(self.n)
        for i,s in enumerate(self.state):
            self.board[s][i] = "Q"
        self.available[:,:] = "X"
        self.allowed_moves = list(zip(*np.where(self.available == "O")))
        return self.print_board()
    
    def random_movement(self):
        """
        Place a queen at random.
        """
        if len(self.allowed_moves)==0:
            return "No more moves allowed!"
        else:
            x,y = random.choice(self.allowed_moves)
            self.put_queen(y,x)
            
    def sequential_movement(self):
        """
        Place the next available queen.
        """
        if len(self.allowed_moves)==0:
            return "No more moves allowed!"
        else:
            x,y = self.allowed_moves[0]
            self.put_queen(y,x)
            
    def play(self,mode):
        '''
        Given a mode, place queens until there are not more options.
        '''
        self.restart()
        print("*** NEW GAME ***")
        while len(self.allowed_moves)!=0:
            mode()
            print()
        return game.is_sol()
    
    def restart(self):
        """
        Restart the game.
        """
        self.__init__(self.n)
        

# Environment 
Chess board

In [ ]:
game = Queens(8)
game.print_board()

# State
Placement of the queens

In [ ]:
game.state

# State

In [ ]:
game.random_movement()
game.state

In [ ]:
game.random_movement()
game.state

# Action
Coordinates of the next queen to place

In [ ]:
game.print_board()

In [ ]:
game.print_available()

In [ ]:
game.allowed_moves

# Reward
1 for a valid placement, 0 for a non-valid placement

In [ ]:
game.is_sol()

# 👾 Let's play (and learn)!

<br>

<center><img src="https://media1.tenor.com/images/1f84b096cbe1cc9f3763c803bb17e10e/tenor.gif?itemid=5878976">

# 👾 Let's play (and learn)!

• Two strategies: 
    - Random placement
    - Sequential placement

In [ ]:
def strategies(trials=1_000):
    strategies = [game.random_movement, game.sequential_movement] 
    outcomes = []
    for strategy in strategies:
        outcomes.append(sum([game.play(strategy) for _ in range(trials)]))
    return outcomes,strategies

def play_and_learn(n=4, trials=1_000):
    game = Queens(n)
    outcomes,st = strategies(trials)
    result = f"The best strategy is: {str(st[outcomes.index(max(outcomes))])[21:].split()[0]} with {max(outcomes)} points in {trials} trials!"
    return outcomes,st,result

In [ ]:
outcomes,strategies,result = play_and_learn()

In [ ]:
print(result)

# 🤔 Reflection

• Random placement performs better than sequential placement. 

• There is a reason this problem is usually solved with Backtracking.

• What would happen if we mix backtracking + reinforcement learning? 🤯

# 📚 References

• Lim, S., Son, K., Park, S., & Lee, S. (2005). The Improvement of Convergence Rate in n-Queen Problem Using Reinforcement learning. Journal of Korean Institute of Intelligent Systems.

• Preußer, T. B., & Engelhardt, M. R. (2016). Putting Queens in Carry Chains, N o?27. Journal of Signal Processing Systems, 88(2), 185-201. doi:10.1007/s11265-016-1176-8

• Russell, S. J., & Norvig, P. (2018). Artificial intelligence a modern approach. Boston: Pearson.

• Spiering, B. (2019). Reinforcement Learning course @ USF. Retrieved from https://github.com/brianspiering/rl-course